In [26]:
import tensorflow as tf
import keras_tuner
import numpy as np
import os
import pathlib
import matplotlib.pyplot as plt
import IPython
import sys
import cv2
import csv


In [27]:
# Dataset loading

data_path = pathlib.Path('/home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/train/')

data_path_test = pathlib.Path('/home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/test/')
AUTOTUNE=tf.data.AUTOTUNE

dataset_path, dataset_path_val = tf.keras.utils.image_dataset_from_directory(      # Training training and validation datasets
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='both',
    seed= 1,
    batch_size=16,
    image_size=(244, 244),
    color_mode="rgb",
    shuffle=True)

dataset_path_test = tf.keras.utils.image_dataset_from_directory(      # Training training and validation datasets
    data_path_test,
    labels= 'inferred',
    seed= 2,
    batch_size=1,
    image_size=(244, 244),
    color_mode="rgb",
    shuffle=True)

Found 900 files belonging to 6 classes.
Using 720 files for training.
Using 180 files for validation.


Found 480 files belonging to 1 classes.


In [28]:
# downloading the weights oft the base model

base_model = tf.keras.applications.InceptionV3(
    input_shape = (244, 244, 3),
    include_top = False, 
    weights = "imagenet"                  
)

In [29]:
#saving the downloaded base_model 

saving_path = pathlib.Path('SavedBaseModel.h5')

base_model.save(saving_path)

In [30]:
#Model

model_path = pathlib.Path('SavedBaseModel.h5')

base_model = tf.keras.models.load_model(model_path)                 #Loading base_model

base_model.trainable = False                                        # Setting the model as non-trainable

rescaling = tf.keras.Sequential([                                        
  tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)             # Rescaling to (1, -1) range required for inceptionV3 model
])
augmentation = tf.keras.Sequential([                                # Applying augmentations on the images
   tf.keras.layers.RandomFlip("horizontal"),
   tf.keras.layers.RandomRotation(0.1)
])

# def build_model(hp):

inputs = tf.keras.Input(shape=(244, 244, 3))
# x = rescaling(inputs)
# x = augmentation(x)
x = tf.keras.applications.inception_v3.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(6, activation='softmax')(x)                   # "softmax" in the final layer for decision making
model = tf.keras.Model(inputs, outputs)

#   learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00029979988271190114),        # compiling with low learning rate lr=0.00029979988271190114
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)                                               

  # return model 

# build_model(keras_tuner.HyperParameters())

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="val_loss",
#     max_trials=6,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="/home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/",
#     project_name="Tuner",
# )

# tuner.search_space_summary()

# tuner.search(dataset_path, epochs=20, validation_data = dataset_path_val)

# tuner.results_summary()


model.fit(                                                               # fitting the whole model for non-trainable base
    dataset_path,
    epochs=15,   
    validation_data = dataset_path_val)


model.summary()

base_model.trainable = True                                           # switching the base_model to trainable

for layer in base_model.layers:                                       # Swtching all batchnormalization layer of the base_model to non-trainable to not lose weights
  if isinstance(layer, tf.keras.layers.BatchNormalization):
    layer.trainable = False

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000001),    # compiling the model with a trainable base_model
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)

model.fit(                                                            # fitting for another 10 epochs starting where the non-trainable ended
    dataset_path,
    epochs=50,         
    initial_epoch=15,                
    validation_data = dataset_path_val)

# model.evaluate(dataset_path_test, batch_size=5, verbose=1)

model.summary()


Epoch 1/15
23/45 [==============>...............] - ETA: 4s - loss: 1.7127 - accuracy: 0.3016

In [ ]:
# save model

saving_path = pathlib.Path('/home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/saved_model/')

tf.keras.models.save_model(model,
                           saving_path,
                           overwrite=True,
                           save_format='tf'
                           )

INFO:tensorflow:Assets written to: /home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/saved_model/assets


INFO:tensorflow:Assets written to: /home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/saved_model/assets


In [ ]:
# load and compile prediction model

loading_path = pathlib.Path('/home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/saved_model/')
loaded_model = tf.keras.models.load_model(loading_path, compile=True)


loaded_model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits= True),
    metrics=['accuracy'])


In [ ]:
# a function to write to csv files
def csv_writer(filename, fields, data):
  
    csvfile = filename
    fields = fields
    with open(csvfile, mode="a") as first:
        csvwriter = csv.writer(first)
        csvwriter.writerow(fields)
        csvwriter.writerows(data)

        return True

In [ ]:
# file creation for the predicted labels list

submit = []
pred_data_path = pathlib.Path("/home/samer/Desktop/Beedoo/ML_Olympiad/mlo2024mlact/MLAct-MLO2024-Dataset/test/")
for dir1 in os.listdir(pred_data_path):
    for file in os.listdir(os.path.join(pred_data_path, dir1)):
        # print(file)
        image_path = os.path.join(pred_data_path, dir1, file)
        image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
        image = np.array(image)
        image = image.astype("float32")
        pred_image = np.reshape(image, (1,244,244,3))
        predictions = loaded_model.predict(pred_image)
        pred = np.argmax(predictions, axis=1)
        print("this is pred: ", pred)
        if pred == 0:
            PredictedClass = "'barrel_jellyfish'"
            print("barrel_jellyfish")
        elif pred == 1:
            PredictedClass = "'compass_jellyfish'"
            print("compass_jellyfish")
        elif pred == 2:
            PredictedClass = "'lions_mane_jellyfish'"
            print("lions_mane_jellyfish")
        elif pred == 3:
            PredictedClass = "'mauve_stinger_jellyfish'"
            print("mauve_stinger_jellyfish")
        elif pred == 4:
            PredictedClass = "'moon_jellyfish'"
            print("moon_jellyfish")
        elif pred == 5:
            PredictedClass = "'plastic_pollution'"
            print("plastic_pollution")
        # plt.imshow(image.astype("uint8"))
        # plt.show()
        file = file.split('.')[0]
        submit.append((file, PredictedClass))
fields = ["ImageID", 'PredictedClass']
csv_writer("TurtleVisionChallenge_Predictions_fineTuned.csv", fields, submit)

# a csv file contains the predicted classes for the test set of the TurtleVisionChallenge dataset.

1/1 [==============================] - 2s 2s/step
this is pred:  [0]
barrel_jellyfish
1/1 [==============================] - 0s 18ms/step
this is pred:  [5]
plastic_pollution
1/1 [==============================] - 0s 20ms/step
this is pred:  [3]
mauve_stinger_jellyfish
1/1 [==============================] - 0s 17ms/step
this is pred:  [3]
mauve_stinger_jellyfish
1/1 [==============================] - 0s 21ms/step
this is pred:  [4]
moon_jellyfish
1/1 [==============================] - 0s 19ms/step
this is pred:  [2]
lions_mane_jellyfish
1/1 [==============================] - 0s 18ms/step
this is pred:  [4]
moon_jellyfish
1/1 [==============================] - 0s 18ms/step
this is pred:  [4]
moon_jellyfish
1/1 [==============================] - 0s 18ms/step
this is pred:  [0]
barrel_jellyfish
1/1 [==============================] - 0s 19ms/step
this is pred:  [2]
lions_mane_jellyfish
1/1 [==============================] - 0s 17ms/step
this is pred:  [2]
lions_mane_jellyfish
1/1 [========

True